In [29]:
import requests
import json
from pprint import pprint
import pandas as pd
import spotipy as sp
import time

#import spotify keys
from config import client_id
from config import client_secret

In [32]:
#generate access token for spotify api
#from: https://stmorse.github.io/journal/spotify-api.html

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [33]:
# print(access_token)

In [34]:
#from: https://stmorse.github.io/journal/spotify-api.html

#base URL for Spotify API search
search_url = 'https://api.spotify.com/v1/search'

#In order to access, we send a properly formed GET request to the API server, with our access_token in the header. Let’s save this header info now, in the following very specific format:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [52]:
# need to add instructions or code to remove punctuations for search to work
search_track = input("Please name a song: ")

Please name a song:  dont stop me now


In [53]:
search_artist = input("Please name that song's artist: ")

Please name that song's artist:  queen


In [54]:
response = requests.get(search_url + '?q=track%3A' + search_track + '%20artist%3A' + search_artist + '&type=track%2Cartist&limit=1', headers=headers).json()

In [55]:
pprint(response)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=track%3Adont+stop+me+now+artist%3Aqueen&type=artist&offset=0&limit=1',
             'items': [],
             'limit': 1,
             'next': None,
             'offset': 0,
             'previous': None,
             'total': 0},
 'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3Adont+stop+me+now+artist%3Aqueen&type=track&offset=0&limit=1',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1dfeR4HaWDbWqFHLkxsg1d'},
                                              'href': 'https://api.spotify.com/v1/artists/1dfeR4HaWDbWqFHLkxsg1d',
                                              'id': '1dfeR4HaWDbWqFHLkxsg1d',
                                              'name': 'Queen',
                                              'type': 'artist',
                                              'uri': 'spotify:art

In [56]:
pprint(response['tracks']['items'][0]["id"])

'7hQJA50XrCWABAu5v6QZ4i'


In [57]:
track_id = response['tracks']['items'][0]["id"]

In [58]:
#base URL for Spotify API
base_url = 'https://api.spotify.com/v1/'

track_response = requests.get(base_url + 'audio-features/' + track_id, headers=headers).json()

In [59]:
#empty lists to store song data
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
id_num = []
track_href = []
analysis_url = []
duration_mins = []
time_signature = []

In [60]:
#fill in track info for each audio feature / empty list
danceability.append(track_response["danceability"])
energy.append(track_response["energy"])
key.append(track_response["key"])
loudness.append(track_response["loudness"])
mode.append(track_response["mode"])
speechiness.append(track_response["speechiness"])
acousticness.append(track_response["acousticness"])
instrumentalness.append(track_response["instrumentalness"])
liveness.append(track_response["liveness"])
valence.append(track_response["valence"])
tempo.append(track_response["tempo"])
id_num.append(track_response["id"])
track_href.append(track_response["track_href"])
analysis_url.append(track_response["analysis_url"])    
#duration recorded in api in milliseconds, converting to minutes here
duration_mins.append(round((track_response["duration_ms"] / 60000),2))
time_signature.append(track_response["time_signature"])    

In [61]:
#create a dictionary to hold data gathered from the api
track_dict = {
    "track_id": id_num,
    "danceability": danceability,
    "energy": energy,
    "key": key,
    "loudness": loudness,
    "mode": mode,
    "speechiness": speechiness,
    "acousticness": acousticness,
    "instrumentalness": instrumentalness,
    "liveness": liveness,
    "valence": valence,
    "tempo": tempo,
    "duration_mins": duration_mins,
    "time_signature": time_signature,
    "track_href": track_href,
    "analysis_url": analysis_url
    
}
#convert dictionary to a dataframe
input_track_df = pd.DataFrame(track_dict)
input_track_df.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature,track_href,analysis_url
0,7hQJA50XrCWABAu5v6QZ4i,0.563,0.865,5,-5.277,1,0.16,0.0472,0.000191,0.77,0.601,156.271,3.49,4,https://api.spotify.com/v1/tracks/7hQJA50XrCWA...,https://api.spotify.com/v1/audio-analysis/7hQJ...


In [62]:
#import global top songs csv and save as a df
global2022_df = pd.read_csv("resources/regional-global-weekly-2022-10-06.csv")
global2022_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams
0,1,spotify:track:3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),EMI,1,1,2,53478156
1,2,spotify:track:4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Parlophone UK,2,2,6,33516367
2,3,spotify:track:4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Columbia,1,3,27,32910077
3,4,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",DALE PLAY Records,1,4,13,31691825
4,5,spotify:track:5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Sony Music Latin/La Industria,5,5,14,29821181


In [63]:
#extracting track id (removing "spotify:track:" from track id in csv)
track_id_series = global2022_df.uri.str[14:]

#convert to df and rename column
track_id_df = track_id_series.to_frame().rename(columns={"uri":"track_id"})

#merge dfs
global2022_df = global2022_df.merge(track_id_df, how="inner", left_index=True, right_index=True)
global2022_df = global2022_df[["track_id", "artist_names", "track_name"]]
global2022_df = global2022_df.astype({'track_id':'string','artist_names':'string','track_name':'string'})

In [64]:
global2022_df.head()

,track_id,artist_names,track_name
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras)
1,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue)
2,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was
3,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52"
4,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata


In [ ]:
#empty lists to store song data
global_danceability = []
global_energy = []
global_key = []
global_loudness = []
global_mode = []
global_speechiness = []
global_acousticness = []
global_instrumentalness = []
global_liveness = []
global_valence = []
global_tempo = []
global_id_num = []
global_track_href = []
global_analysis_url = []
global_duration_mins = []
global_time_signature = []

#loop through dataframe, use track id to look up songs in spotify api and grab each one's audio features
for t in global2022_df["track_id"]:
    global_response = requests.get(base_url + 'audio-features/' + t, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    global_danceability.append(global_response["danceability"])
    global_energy.append(global_response["energy"])
    global_key.append(global_response["key"])
    global_loudness.append(global_response["loudness"])
    global_mode.append(global_response["mode"])
    global_speechiness.append(global_response["speechiness"])
    global_acousticness.append(global_response["acousticness"])
    global_instrumentalness.append(global_response["instrumentalness"])
    global_liveness.append(global_response["liveness"])
    global_valence.append(global_response["valence"])
    global_tempo.append(global_response["tempo"])
    global_id_num.append(global_response["id"])
    global_track_href.append(global_response["track_href"])
    global_analysis_url.append(global_response["analysis_url"])    
    #duration recorded in api in milliseconds, converting to minutes here
    global_duration_mins.append(round((global_response["duration_ms"] / 60000),2))
    global_time_signature.append(global_response["time_signature"])   
    
#create a dictionary to hold data gathered from the api
global_songs_dict = {
    "track_id": global_id_num,
    "danceability": global_danceability,
    "energy": global_energy,
    "key": global_key,
    "loudness": global_loudness,
    "mode": global_mode,
    "speechiness": global_speechiness,
    "acousticness": global_acousticness,
    "instrumentalness": global_instrumentalness,
    "liveness": global_liveness,
    "valence": global_valence,
    "tempo": global_tempo,
    "duration_mins": global_duration_mins,
    "time_signature": global_time_signature,
    "track_href": global_track_href,
    "analysis_url": global_analysis_url
    
}
#convert dictionary to a dataframe
gloabl_info_df = pd.DataFrame(global_songs_dict)
gloabl_info_df.head()

In [ ]:
global_df = global2022_df.merge(gloabl_info_df, how="inner")
global_df

In [ ]:
#import regional top songs csvs and save as individual dfs
regional_files = {"usa":"regional-usa-weekly-2022-10-06.csv", "vietnam":"regional-vietnam-weekly-2022-10-06.csv","venezuela":"regional-venezuela-weekly-2022-10-06.csv",
"uruguay":"regional-uruguay-weekly-2022-10-06.csv","united_kingdom":"regional-united_kingdom-weekly-2022-10-06.csv","ukraine":"regional-ukraine-weekly-2022-10-06.csv",
"uae":"regional-uae-weekly-2022-10-06.csv","turkey":"regional-turkey-weekly-2022-10-06.csv","thailand":"regional-thailand-weekly-2022-10-06.csv",
"taiwan":"regional-taiwan-weekly-2022-10-06.csv","switzerland":"regional-switzerland-weekly-2022-10-06.csv","sweden":"regional-sweden-weekly-2022-10-06.csv",
"spain":"regional-spain-weekly-2022-10-06.csv","south_korea":"regional-south_korea-weekly-2022-10-06.csv","south_africa":"regional-south_africa-weekly-2022-10-06.csv",
"slovakia":"regional-slovakia-weekly-2022-10-06.csv","singapore":"regional-singapore-weekly-2022-10-06.csv","saudi_arabia":"regional-saudi_arabia-weekly-2022-10-06.csv",
"romania":"regional-romania-weekly-2022-10-06.csv","portugal":"regional-portugal-weekly-2022-10-06.csv","poland":"regional-poland-weekly-2022-10-06.csv",
"philippines":"regional-philippines-weekly-2022-10-06.csv","peru":"regional-peru-weekly-2022-10-06.csv","paraguay":"regional-paraguay-weekly-2022-10-06.csv",
"panama":"regional-panama-weekly-2022-10-06.csv","pakistan":"regional-pakistan-weekly-2022-10-06.csv","norway":"regional-norway-weekly-2022-10-06.csv",
"nigeria":"regional-nigeria-weekly-2022-10-06.csv","nicaragua":"regional-nicaragua-weekly-2022-10-06.csv","new_zealand":"regional-new_zealand-weekly-2022-10-06.csv",
"netherlands":"regional-netherlands-weekly-2022-10-06.csv","morocco":"regional-morocco-weekly-2022-10-06.csv","mexico":"regional-mexico-weekly-2022-10-06.csv",
"malaysia":"regional-malaysia-weekly-2022-10-06.csv","luxembourg":"regional-luxembourg-weekly-2022-10-06.csv","lithuania":"regional-lithuania-weekly-2022-10-06.csv",
"latvia":"regional-latvia-weekly-2022-10-06.csv","kazakhstan":"regional-kazakhstan-weekly-2022-10-06.csv","japan":"regional-japan-weekly-2022-10-06.csv",
"italy":"regional-italy-weekly-2022-10-06.csv","israel":"regional-israel-weekly-2022-10-06.csv","ireland":"regional-ireland-weekly-2022-10-06.csv",
"indonesia":"regional-indonesia-weekly-2022-10-06.csv","india":"regional-india-weekly-2022-10-06.csv","iceland":"regional-iceland-weekly-2022-10-06.csv",
"hungary":"regional-hungary-weekly-2022-10-06.csv","hong_kong":"regional-hong_kong-weekly-2022-10-06.csv","honduras":"regional-honduras-weekly-2022-10-06.csv",
"guatemala":"regional-guatemala-weekly-2022-10-06.csv","greece":"regional-greece-weekly-2022-10-06.csv","germany":"regional-germany-weekly-2022-10-06.csv",
"france":"regional-france-weekly-2022-10-06.csv","finland":"regional-finland-weekly-2022-10-06.csv","estonia":"regional-estonia-weekly-2022-10-06.csv",
"el_salvador":"regional-el_salvador-weekly-2022-10-06.csv","egypt":"regional-egypt-weekly-2022-10-06.csv","ecuador":"regional-ecuador-weekly-2022-10-06.csv",
"dominican_republic":"regional-dominican_republic-weekly-2022-10-06.csv","denmark":"regional-denmark-weekly-2022-10-06.csv","czech_republic":"regional-czech_republic-weekly-2022-10-06.csv",
"cyprus":"regional-cyprus-weekly-2022-10-06.csv","costa_rica":"regional-costa_rica-weekly-2022-10-06.csv","colombia":"regional-colombia-weekly-2022-10-06.csv",
"chile":"regional-chile-weekly-2022-10-06.csv","canada":"regional-canada-weekly-2022-10-06.csv","bulgaria":"regional-bulgaria-weekly-2022-10-06.csv",
"brazil":"regional-brazil-weekly-2022-10-06.csv","bolivia":"regional-bolivia-weekly-2022-10-06.csv","belgium":"regional-belgium-weekly-2022-10-06.csv",
"belarus":"regional-belarus-weekly-2022-10-06.csv","austria":"regional-austria-weekly-2022-10-06.csv","australia":"regional-australia-weekly-2022-10-06.csv",
"argentina":"regional-argentina-weekly-2022-10-06.csv"}

d = {}

for key in regional_files:
    # print(key, regional_files[key])
    d[key] = pd.read_csv(f'resources/{regional_files[key]}') 
    
    #extracting track id (removing "spotify:track:" from track id in csv)
    track_id_series = d[key].uri.str[14:]
    
    #convert to df and rename column
    track_id_df = track_id_series.to_frame().rename(columns={"uri":"track_id"})
    
    #merge dfs
    d[key] = d[key].merge(track_id_df, how="inner", left_index=True, right_index=True)
    d[key] = d[key][["track_id", "artist_names", "track_name"]]
    #add column for region
    d[key]['region'] = key.capitalize()
    #convert values to string
    d[key] =d[key].astype({'track_id':'string','artist_names':'string','track_name':'string','region':'string'})

In [25]:
d['argentina']

,track_id,artist_names,track_name,region
0,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Argentina
1,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Argentina
2,45dlaEXrXW47TVbrQ4gK7E,"Rei, Callejero Fino",Tu Turrito,Argentina
3,3Ne5uYDFzHEFYAPbaBMIck,Maria Becerra,AUTOMÁTICO,Argentina
4,53tfEupEzQRtVFOeZvk7xq,ROSALÍA,DESPECHÁ,Argentina
...,...,...,...,...
195,6nAlJuLlRlpAcq30xIqKJs,Plan B,¿Por Que Te Demoras?,Argentina
196,4iaNmAvcZ6sPGvgZ6vQQdx,"Quevedo, Linton",Ahora y Siempre,Argentina
197,65DBZofI0b79kfHTcWWDuU,Soda Stereo,Trátame Suavemente - Remasterizado 2007,Argentina
198,0WuKq2LiraBFney78dzwoc,Ciro y los Persas,Mírenla,Argentina


In [26]:
d['argentina'].dtypes

track_id        string
artist_names    string
track_name      string
region          string
dtype: object

In [27]:
regional_list = d.keys()
regional_list

dict_keys(['usa', 'vietnam', 'venezuela', 'uruguay', 'united_kingdom', 'ukraine', 'uae', 'turkey', 'thailand', 'taiwan', 'switzerland', 'sweden', 'spain', 'south_korea', 'south_africa', 'slovakia', 'singapore', 'saudi_arabia', 'romania', 'portugal', 'poland', 'philippines', 'peru', 'paraguay', 'panama', 'pakistan', 'norway', 'nigeria', 'nicaragua', 'new_zealand', 'netherlands', 'morocco', 'mexico', 'malaysia', 'luxembourg', 'lithuania', 'latvia', 'kazakhstan', 'japan', 'italy', 'israel', 'ireland', 'indonesia', 'india', 'iceland', 'hungary', 'hong_kong', 'honduras', 'guatemala', 'greece', 'germany', 'france', 'finland', 'estonia', 'el_salvador', 'egypt', 'ecuador', 'dominican_republic', 'denmark', 'czech_republic', 'cyprus', 'costa_rica', 'colombia', 'chile', 'canada', 'bulgaria', 'brazil', 'bolivia', 'belgium', 'belarus', 'austria', 'australia', 'argentina'])

In [28]:
len(regional_list)

73

In [30]:
f = {}

In [31]:
for name, df in d.items():
    items = df["track_id"]
    # print('next')

    for i in items:
        # print(i)

        regional_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()

        # operate on DataFrame 'df' for company 'name'
        #empty lists to store song data
        danceability = []
        energy = []
        key = []
        loudness = []
        mode = []
        speechiness = []
        acousticness = []
        instrumentalness = []
        liveness = []
        valence = []
        tempo = []
        id_num = []
        track_href = []
        analysis_url = []
        duration_mins = []
        time_signature = []
    
        #fill in track info for each audio feature / empty list
        danceability.append(regional_response["danceability"])
        energy.append(regional_response["energy"])
        key.append(regional_response["key"])
        loudness.append(regional_response["loudness"])
        mode.append(regional_response["mode"])
        speechiness.append(regional_response["speechiness"])
        acousticness.append(regional_response["acousticness"])
        instrumentalness.append(regional_response["instrumentalness"])
        liveness.append(regional_response["liveness"])
        valence.append(regional_response["valence"])
        tempo.append(regional_response["tempo"])
        id_num.append(regional_response["id"])
        track_href.append(regional_response["track_href"])
        analysis_url.append(regional_response["analysis_url"])    
        #duration recorded in api in milliseconds, converting to minutes here
        duration_mins.append(round((regional_response["duration_ms"] / 60000),2))
        time_signature.append(regional_response["time_signature"])  

        time.sleep(1)
        
        regional_dict = {}
        #create a dictionary to hold data gathered from the api
        regional_dict = {
            "track_id": id_num,
            "danceability": danceability,
            "energy": energy,
            "key": key,
            "loudness": loudness,
            "mode": mode,
            "speechiness": speechiness,
            "acousticness": acousticness,
            "instrumentalness": instrumentalness,
            "liveness": liveness,
            "valence": valence,
            "tempo": tempo,
            "duration_mins": duration_mins,
            "time_signature": time_signature,
            "track_href": track_href,
            "analysis_url": analysis_url
        }

        #convert dictionary to a dataframe
        f[name] = pd.DataFrame(regional_dict)
    

KeyError: 'danceability'